In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from humanize import naturaldelta
from matplotlib.patches import Patch

sys.path.insert(0, "..")
from runner.utils import (
    allocate_benchmarks,
    create_benchmark_campaign,
    load_benchmark_metadata,
)

In [ ]:
# If a util function was modified, use this cell to reload it without having to restart the kernel
%run ../runner/utils.py

### Util to load results from older runs

In [58]:
# Use this function for 20251128 and 20251201 since those runs were before we started recording `VM Zone`, which is required by utils.load_results
def load_results(folder: str | list[str]):
    """Loads all CSV files in `folder`. Returns the results and variability dataframes."""
    folders = folder if isinstance(folder, list) else [folder]
    csv_files = [p for f in folders for p in Path(f).glob("*.csv")]
    results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)

    # Remove reference benchmark
    reference_results = results.query('Benchmark == "reference-benchmark"')
    results = results.query('Benchmark != "reference-benchmark"').copy()

    # Find the variability of each VM
    variability = reference_results.groupby(["Hostname", "Run ID"]).agg(
        {"Runtime (s)": ["count", "min", "max", "std", "mean"]}
    )
    variability["std %"] = (
        variability[("Runtime (s)", "std")] * 100 / variability[("Runtime (s)", "mean")]
    )

    # Print some basic stats
    results["bench-size"] = results["Benchmark"] + "-" + results["Size"]
    results["solver-version"] = results["Solver"] + "-" + results["Solver Version"]
    print(
        f"Found {len(results)} records, {len(results['bench-size'].unique())} benchmark instances"
    )
    return results, variability

## Setup benchmark campaign

### 20251128 Test HiPO on paper's PyPSA instances

In [ ]:
urls = """https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-10-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-2-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-3-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-4-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-5-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-6-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-7-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-8-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-op-9-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-10-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-2-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-3-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-4-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-5-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-6-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-7-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-8-3h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-12h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-1h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-24h.lp
https://storage.googleapis.com/solver-benchmarks/pypsa-eur-elec-trex-9-3h.lp"""

benchmarks = {}
for url in urls.strip().split("\n"):
    # Extract filename from URL
    filename = url.split("/")[-1].replace(".lp", "")
    # Parse benchmark name and size (first 4 parts as benchmark name, rest as size name)
    parts = filename.split("-")
    if len(parts) >= 5:
        key = "-".join(parts[:4])
        name = "-".join(parts[4:])
        if key not in benchmarks:
            benchmarks[filename] = {"Sizes": []}
        benchmarks[filename]["Sizes"].append({"Name": name, "Size": None, "URL": url})

# Write to yaml file
with open("../pypsa-hipo-benchmarks.yaml", "w") as f:
    yaml.dump({"benchmarks": benchmarks}, f, default_flow_style=False, sort_keys=False)

In [12]:
with open("../pypsa-hipo-benchmarks.yaml", "r") as f:
    hipo_benchmarks = yaml.safe_load(f)

In [ ]:
def get_machine_type(size):
    return "c4-highmem-8" if size == "L" else "c4-standard-2"

In [ ]:
# Create hipo benchmarks

# Create output directory for HIPO benchmarks
hipo_output_dir = Path("../infrastructure/benchmarks/20251128-test-hipo")
hipo_output_dir.mkdir(parents=True, exist_ok=True)

# Check for existing yaml files and clean them up
existing_files = list(hipo_output_dir.glob("*.yaml"))
if existing_files:
    print(
        f"Warning: Found {len(existing_files)} existing yaml files in {hipo_output_dir}"
    )
    for f in existing_files:
        f.unlink()
    print("Cleaned up existing files.")

# Pick the largest 4 instances of each bench
op_benchs = [
    (k, v)
    for (k, v) in hipo_benchmarks.get("benchmarks", {}).items()
    if k.startswith("pypsa-eur-elec-op")
]
op_benchs = sorted(op_benchs, key=lambda x: x[1]["Sizes"][0]["Num. variables"])[-4:]
tr_benchs = [
    (k, v)
    for (k, v) in hipo_benchmarks.get("benchmarks", {}).items()
    if k.startswith("pypsa-eur-elec-trex")
]
tr_benchs = sorted(tr_benchs, key=lambda x: x[1]["Sizes"][0]["Num. variables"])[-4:]

# Process each benchmark from the hipo benchmarks file
hipo_idx = 0
for benchmark_name, benchmark_data in op_benchs + tr_benchs:
    # Remove the size name from the benchmark name
    benchmark_name = "-".join(benchmark_name.split("-")[:-2])
    for size_data in benchmark_data.get("Sizes", []):
        size = size_data["Size"]
        # Create output filename
        output_filename = f"{hipo_idx:02d}-pypsa-hipo.yaml"
        output_path = hipo_output_dir / output_filename

        # Build the output yaml structure
        output_yaml = {
            "machine-type": get_machine_type(size),
            "zone": "us-central1-a",
            "years": [2025],
            "solver": "highs-hipo-ipm highs-hipo-64 highs-hipo-no2hop",
            # 'timeout_seconds': 24*60*60,
            "benchmarks": {
                benchmark_name: {
                    "Sizes": [
                        {
                            "Name": size_data["Name"],
                            "Size": size,
                            "URL": size_data["URL"],
                        }
                    ]
                }
            },
        }

        # Write to file
        with open(output_path, "w") as f:
            yaml.dump(output_yaml, f, default_flow_style=False, sort_keys=False)

        print(f"Created: {output_filename}")
        hipo_idx += 1

print(f"\nTotal HIPO files created: {len(list(hipo_output_dir.glob('*.yaml')))}")

Cleaned up existing files.
Created: 00-pypsa-hipo.yaml
Created: 01-pypsa-hipo.yaml
Created: 02-pypsa-hipo.yaml
Created: 03-pypsa-hipo.yaml
Created: 04-pypsa-hipo.yaml
Created: 05-pypsa-hipo.yaml
Created: 06-pypsa-hipo.yaml
Created: 07-pypsa-hipo.yaml

Total HIPO files created: 8


In [20]:
with open("../pypsa-hipo-benchmarks.yaml", "r") as f:
    hipo_benchmarks = yaml.safe_load(f)

### 20251201 Test new large PyPSA instances

In [16]:
# Load the new benchmarks
with open("../benchmarks/pypsa/metadata_new_pypsa_benchmarks.yaml", "r") as f:
    new_benchmarks = yaml.safe_load(f)

In [ ]:
# Create benchmark campaign
output_dir = Path("../infrastructure/benchmarks/20251201-hipo-new-pypsa")
output_dir.mkdir(parents=True, exist_ok=True)


# Determine machine type and solvers based on size and problem class
def get_machine_type(size):
    return "c4-highmem-8" if size == "L" else "c4-standard-2"


idx = 0
for benchmark_name, benchmark_data in new_benchmarks["benchmarks"].items():
    for size_data in benchmark_data.get("Sizes", []):
        size = size_data["Size"]

        output_filename = f"{idx:02d}-hipo-new-pypsa.yaml"
        output_path = output_dir / output_filename
        output_yaml = {
            "machine-type": get_machine_type(size),
            "zone": "us-central1-a",
            "years": [2025],
            "solver": "gurobi highs-hipo-no2hop highs-hipo-64 highs-hipo-ipm",
            "timeout_seconds": 24 * 60 * 60,
            "benchmarks": {
                benchmark_name: {
                    "Sizes": [
                        {
                            "Name": size_data["Name"],
                            "Size": size,
                            "URL": size_data["URL"],
                        }
                    ]
                }
            },
        }

        with open(output_path, "w") as f:
            yaml.dump(output_yaml, f, default_flow_style=False, sort_keys=False)
        print(f"Created: {output_filename}")
        idx += 1

print(f"\nTotal files created: {len(list(output_dir.glob('*.yaml')))}")

Created: 00-hipo-new-pypsa.yaml
Created: 01-hipo-new-pypsa.yaml
Created: 02-hipo-new-pypsa.yaml
Created: 03-hipo-new-pypsa.yaml
Created: 04-hipo-new-pypsa.yaml
Created: 05-hipo-new-pypsa.yaml
Created: 06-hipo-new-pypsa.yaml
Created: 07-hipo-new-pypsa.yaml

Total files created: 8


In [19]:
num_vars_and_bench = []
for bench, meta in new_benchmarks["benchmarks"].items():
    for size in meta["Sizes"]:
        num_vars_and_bench.append((size["Num. variables"], f"{bench}-{size['Name']}"))
for num_vs, bench in sorted(num_vars_and_bench):
    print(f"{bench}: {num_vs:,.0f}")

pypsa-eur-sec-trex_vopt-100-3h: 0
pypsa-eur-elec-trex_vopt-50-24h: 477,803
pypsa-eur-elec-trex_vopt-50-12h: 954,858
pypsa-eur-elec-trex_vopt-100-12h: 1,829,368
pypsa-eur-elec-trex_vopt-50-8h: 1,829,368
pypsa-eur-elec-trex_vopt-100-8h: 2,743,328
pypsa-eur-elec-trex_vopt-50-3h: 3,817,188
pypsa-eur-elec-trex_vopt-50-1h: 11,450,068
pypsa-eur-sec-trex_vopt-50-3h: 18,146,578
pypsa-eur-elec-trex_vopt-100-1h: 21,936,488
pypsa-eur-sec-trex_vopt-50-1h: 54,430,498


In [ ]:
# Check the results manually (can't use utils.load_results because this run didn't record `VM Zone1`)
csv_files = [
    p for p in Path("../results/gcp-results/20251201-hipo-new-pypsa/").glob("*.csv")
]
results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)
reference_results = results.query('Benchmark == "reference-benchmark"')
results = results.query('Benchmark != "reference-benchmark"').copy()
results

/var/folders/mb/7xpzpvcd0px0nn2skl6rt8nm0000gn/T/ipykernel_58995/3474435443.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,Max Integrality Violation,Duality Gap,Reported Runtime (s),Timeout,Hostname,Run ID,Timestamp
0,pypsa-eur-sec-trex_vopt,100-3h,gurobi,12.0.3,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-03-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:21:38.123969
2,pypsa-eur-sec-trex_vopt,100-3h,highs-hipo-no2hop,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-03-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:32:13.029337
3,pypsa-eur-sec-trex_vopt,100-3h,highs-hipo-64,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-03-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:35:31.739984
4,pypsa-eur-sec-trex_vopt,100-3h,highs-hipo-ipm,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-03-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:35:46.736482
5,pypsa-eur-elec-trex_vopt,50-1h,gurobi,12.0.3,2025.0,TO,Timeout,86400.000000,30807.732,NaN,NaN,NaN,86400.000000,86400.0,benchmark-instance-04-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:19:33.382964
7,pypsa-eur-sec-trex_vopt,50-3h,gurobi,12.0.3,2025.0,TO,Timeout,86400.000000,48765.440,NaN,NaN,NaN,86400.000000,86400.0,benchmark-instance-01-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:19:46.293980
9,pypsa-eur-elec-trex_vopt,100-1h,gurobi,12.0.3,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-06-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:20:29.468980
11,pypsa-eur-elec-trex_vopt,100-1h,highs-hipo-no2hop,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-06-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:52:24.189929
12,pypsa-eur-elec-trex_vopt,100-1h,highs-hipo-64,1.12.0-hipo,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-06-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 09:20:58.978782
13,pypsa-eur-sec-trex_vopt,50-1h,gurobi,12.0.3,2025.0,OOM,Out of Memory,NaN,NaN,NaN,NaN,NaN,NaN,86400.0,benchmark-instance-00-hipo-new-pypsa,20251201-hipo-new-pypsa,2025-12-01 08:22:49.404869


Conclusion: most of these are too large! I killed the running VMs so as to not waste compute.

### 20251219 Run all sizes of a new PyPSA bench
Goal: to get a sense of which sizes are solvable

In [9]:
new_pypsa_benchs = load_benchmark_metadata("../benchmarks/pypsa/metadata.yaml")

In [13]:
# Run all de-elec instances > 5e5
to_run = new_pypsa_benchs.query(
    'Benchmark == "pypsa-de-elec" and `Num. variables` > 100000'
)
vm_yamls = allocate_benchmarks(
    to_run, "Num. variables", 11, machine_type="c4-highmem-8"
)  # 1 per VM

Allocated. Estimated runtime: 2399.4h
  VM 00: 1 instances, 2399.4h
  VM 01: 1 instances, 1070.7h
  VM 02: 1 instances, 799.9h
  VM 03: 1 instances, 586.5h
  VM 04: 1 instances, 356.9h
  VM 05: 1 instances, 200.1h
  VM 06: 1 instances, 195.5h
  VM 07: 1 instances, 100.1h
  VM 08: 1 instances, 89.3h
  VM 09: 1 instances, 48.9h
  VM 10: 1 instances, 44.7h


In [15]:
# Only run latest highs variants
for y in vm_yamls:
    y["years"] = [2025]
    y["solver"] = "highs-hipo highs-ipm highs"
create_benchmark_campaign("20251219-pypsa-sizes", "pypsa-sizes", vm_yamls)

Created directory and files in ../infrastructure/benchmarks/20251219-pypsa-sizes
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20251219-pypsa-sizes/run.tfvars -state=states/20251219-pypsa-sizes.tfstate


## Download logs & Results

Results were downloaded by running the following command on the root directory of this repository:
```
gsutil -m rsync -r gs://solver-benchmarks/logs ./runner/logs/ && gsutil -m rsync -r gs://solver-benchmarks-restricted/logs ./runner/logs/  && gsutil -m rsync -r gs://solver-benchmarks/results ./results/gcp-results/
```
Then, the cells in the next section were run to clean, check, and analyze results.

## Analyze Results

In [52]:
# results, variability = load_results(['../results/gcp-results/20251128-test-hipo/', '../results/gcp-results/20251219-pypsa-sizes/'])
results, variability = load_results("../results/gcp-results/20251219-pypsa-sizes/")

Found 33 records, 11 benchmark instances


### Check runtime variability

In [19]:
variability.sort_values(by="std %", ascending=False)

Runtime (s)  \
                                                                           count   
Hostname                          Run ID               VM Zone                     
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-central1-a           2   
benchmark-instance-pypsa-sizes-07 20251219-pypsa-sizes us-central1-a           2   
benchmark-instance-pypsa-sizes-06 20251219-pypsa-sizes us-central1-a           2   
benchmark-instance-pypsa-sizes-08 20251219-pypsa-sizes us-central1-a           1   
benchmark-instance-pypsa-sizes-09 20251219-pypsa-sizes us-central1-a           1   
benchmark-instance-pypsa-sizes-10 20251219-pypsa-sizes us-central1-a           1   

                                                                                  \
                                                                             min   
Hostname                          Run ID               VM Zone                     
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-central1-a  213.517188   
benchmark-instance-pypsa-sizes-07 20251219-pypsa-sizes us-central1-a  211.459494   
benchmark-instance-pypsa-sizes-06 20251219-pypsa-sizes us-central1-a  237.979433   
benchmark-instance-pypsa-sizes-08 20251219-pypsa-sizes us-central1-a  223.262342   
benchmark-instance-pypsa-sizes-09 20251219-pypsa-sizes us-central1-a  221.899480   
benchmark-instance-pypsa-sizes-10 20251219-pypsa-sizes us-central1-a  227.473434   

                                                                                  \
                                                                             max   
Hostname                          Run ID               VM Zone                     
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-central1-a  220.382993   
benchmark-instance-pypsa-sizes-07 20251219-pypsa-sizes us-central1-a  217.769141   
benchmark-instance-pypsa-sizes-06 20251219-pypsa-sizes us-central1-a  242.284721   
benchmark-instance-pypsa-sizes-08 20251219-pypsa-sizes us-central1-a  223.262342   
benchmark-instance-pypsa-sizes-09 20251219-pypsa-sizes us-central1-a  221.899480   
benchmark-instance-pypsa-sizes-10 20251219-pypsa-sizes us-central1-a  227.473434   

                                                                                \
                                                                           std   
Hostname                          Run ID               VM Zone                   
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-central1-a  4.854857   
benchmark-instance-pypsa-sizes-07 20251219-pypsa-sizes us-central1-a  4.461594   
benchmark-instance-pypsa-sizes-06 20251219-pypsa-sizes us-central1-a  3.044298   
benchmark-instance-pypsa-sizes-08 20251219-pypsa-sizes us-central1-a       NaN   
benchmark-instance-pypsa-sizes-09 20251219-pypsa-sizes us-central1-a       NaN   
benchmark-instance-pypsa-sizes-10 20251219-pypsa-sizes us-central1-a       NaN   

                                                                                  \
                                                                            mean   
Hostname                          Run ID               VM Zone                     
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-central1-a  216.950091   
benchmark-instance-pypsa-sizes-07 20251219-pypsa-sizes us-central1-a  214.614318   
benchmark-instance-pypsa-sizes-06 20251219-pypsa-sizes us-central1-a  240.132077   
benchmark-instance-pypsa-sizes-08 20251219-pypsa-sizes us-central1-a  223.262342   
benchmark-instance-pypsa-sizes-09 20251219-pypsa-sizes us-central1-a  221.899480   
benchmark-instance-pypsa-sizes-10 20251219-pypsa-sizes us-central1-a  227.473434   

                                                                         std %  
                                                                                
Hostname                          Run ID               VM Zone                  
benchmark-instance-pypsa-sizes-05 20251219-pypsa-sizes us-c

### Check objective values

In [6]:
# Check that all solvers report similar objective values

# TODO glpk is being ignored here
# obj_dev = results[(~results['Objective Value'].isna())].groupby(['Benchmark', 'Size']).agg({'Objective Value': ['min', 'max', 'std']})
obj_dev = (
    results[(~results["Objective Value"].isna()) & (results["Solver"] != "glpk")]
    .groupby(["Benchmark", "Size"])
    .agg({"Objective Value": ["min", "max", "std"]})
)

obj_dev["std_ratio"] = (
    obj_dev[("Objective Value", "std")] / obj_dev[("Objective Value", "min")]
)
obj_dev[~obj_dev["std_ratio"].isna()].sort_values(by="std_ratio").tail()

Objective Value                              \
                                      min           max           std   
Benchmark           Size                                                
pypsa-eur-elec-op   8-1h     8.639078e+09  8.639078e+09  1.407634e+01   
pypsa-eur-elec-trex 7-1h     8.429437e+09  8.429437e+09  1.571379e+01   
pypsa-eur-elec-op   7-1h     8.589274e+09  8.589274e+09  1.625433e+01   
pypsa-eur-elec-trex 9-1h     8.449221e+09  8.449221e+09  2.124908e+01   
pypsa-eur-elec-op   10-1h    8.577142e+09  8.584748e+09  4.391573e+06   

                              std_ratio  
                                         
Benchmark           Size                 
pypsa-eur-elec-op   8-1h   1.629380e-09  
pypsa-eur-elec-trex 7-1h   1.864157e-09  
pypsa-eur-elec-op   7-1h   1.892399e-09  
pypsa-eur-elec-trex 9-1h   2.514916e-09  
pypsa-eur-elec-op   10-1h  5.120090e-04

### Bar plot of relative slowdowns

Copied this code from v1 Key Insights, but haven't ported it to work with these results yet. We should ideally make this code more robust and put it in utils.py

In [7]:
# TODO make this a util fn

max_num_solvers = 5  # TODO

# Color map
color_map = {
    "cbc": "#F9CD5A",  # yellow
    "glpk": "#7C3AED",  # purple
    "gurobi": "#F66C49",  # red
    "highs": "#43BF94",  # green
    "scip": "#3B82F6",  # blue
    # These are colors for hipo variants TODO get new ones?
    "highs-hipo-ipm": "#43BF94",  # green
    "highs-hipo-64": "#7C3AED",  # purple
    "highs-hipo-no2hop": "#3B82F6",  # blue
}  # TODO change these colors on the website too!


def plot_runtime_slowdowns(df, figsize=(12, 6)):
    """Plots relative runtimes (slowdown factors) in a bar chart.

    Expects df to have columns: Benchmark, Solver, Runtime (s), Status, Timeout
    TODO validate df before plotting?
    """
    if "bench-size" in df.columns:
        df["Benchmark"] = df["bench-size"]
    # Fill NaN runtimes and non-ok statuses with TO value
    df.loc[df["Runtime (s)"].isna(), "Runtime (s)"] = df["Timeout"]
    df.loc[df.query('Status != "ok"').index, "Runtime (s)"] = df["Timeout"]

    # Find the fastest solver for each benchmark
    df_ok = df.query('Status == "ok"')
    fastest_runtimes = df_ok.loc[df_ok.groupby("Benchmark")["Runtime (s)"].idxmin()]

    # Sort benchmarks by fastest runtime
    sorted_benchmarks = fastest_runtimes.sort_values("Runtime (s)").reset_index(
        drop=True
    )

    # Normalize runtimes to find each solver's slowdown factor
    df = df.merge(
        fastest_runtimes[["Benchmark", "Runtime (s)"]],
        on="Benchmark",
        suffixes=("", "_fastest"),
    )
    df["Slowdown"] = df["Runtime (s)"] / df["Runtime (s)_fastest"]
    max_slowdown = max(df.query('Status == "ok"')["Slowdown"])
    df.loc[df.query('Status != "ok"').index, "Slowdown"] = 1.1 * max_slowdown

    width = 1 / (max_num_solvers + 1)  # the width of the bars

    fig, ax = plt.subplots(figsize=figsize, layout="constrained")

    # Add a dotted line at y=1
    ax.axhline(1, color="grey", linestyle="--")

    # Plot each benchmark
    for i, benchmark in enumerate(sorted_benchmarks["Benchmark"]):
        benchmark_data = df[df["Benchmark"] == benchmark].sort_values(
            "Slowdown", ascending=True
        )
        num_solvers = len(benchmark_data)
        # Compute x-axis offsets
        xs = i + (np.arange(num_solvers) * width) - 0.5 + width
        # Pick colors based on solvers
        # colors = [color_map[s] for s in benchmark_data["Solver"]]
        colors = [
            color_map[r["Solver"]]
            if r["Status"] == "ok"
            else (color_map[r["Solver"]], 0.2)
            for _, r in benchmark_data.iterrows()
        ]
        # Push down non-ok bars to not make them distracting
        # benchmark_data.loc[benchmark_data.query('Status != "ok"').index, "Slowdown"] = 1
        ax.bar(xs, benchmark_data["Slowdown"], width, color=colors)
        # Add text labels on top of bars
        for i, x in enumerate(xs):
            if benchmark_data.iloc[i]["Status"] == "ok":
                y = benchmark_data.iloc[i]["Slowdown"] + 0.5
                label = f"{benchmark_data.iloc[i]['Slowdown']:.1f}x"
                kwargs = {}
            else:
                # y = 1.1
                y = benchmark_data.iloc[i]["Slowdown"] + 0.5
                label = benchmark_data.iloc[i]["Status"]
                kwargs = {"color": "red", "weight": "bold"}
            ax.text(
                x,
                y,
                label,
                ha="center",
                **kwargs,
            )

    # Set x-ticks and labels
    xlabels = [
        f"{r['Benchmark']}\nFastest solver: {naturaldelta(r['Runtime (s)'])}"
        for _, r in sorted_benchmarks.iterrows()
    ]
    ax.set_xticks(np.arange(len(sorted_benchmarks)), xlabels)

    # Add labels and title
    ax.set_ylabel("Relative Runtime (normalized)")
    ax.set_title("Solver Runtime Comparison")
    ax.legend(
        handles=[Patch(color=c, label=s) for s, c in color_map.items()],
        title="Solver",
        loc="upper right",
    )


def plot_runtime_slowdowns_of(results, to_plot, figsize=(12, 6)):
    to_plot_results = results[results["bench-size"].isin(to_plot)]

    # Remove old solver versions
    to_plot_results = to_plot_results.sort_values(
        ["bench-size", "Solver", "Solver Release Year"]
    ).drop_duplicates(["bench-size", "Solver"], keep="last")

    plot_runtime_slowdowns(to_plot_results, figsize=figsize)


# TODO for overall plot -- num solved inside bar?
# TODO make non-ok bars more transparent?

In [ ]:
plot_runtime_slowdowns_of(
    results, {b for b in results["bench-size"] if b != "reference-benchmark-reference"}
)

### Performance of HiPO variants on v1 PyPSA benchmarks (also used in HiPO paper)

Run ID: `20251128-test-hipo`

In [60]:
results, variability = load_results(["../results/gcp-results/20251128-test-hipo/"])

Found 24 records, 8 benchmark instances


In [62]:
with open("../pypsa-hipo-benchmarks.yaml", "r") as f:
    hipo_benchmarks = yaml.safe_load(f)

In [ ]:
# Create a speedup dataframe from the results dataframe
speedup_df = results[results["Solver"].str.contains("highs-hipo")].copy()
speedup_df = speedup_df.pivot_table(
    index="bench-size", columns="Solver", values="Runtime (s)", aggfunc="first"
).reset_index()

# Also pivot Status column
status_df = (
    results[results["Solver"].str.contains("highs-hipo")]
    .pivot_table(index="bench-size", columns="Solver", values="Status", aggfunc="first")
    .reset_index()
)

# Calculate speedups relative to ipm-time, but use status if not "ok"
speedup_df["hipo-64-speedup"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo-64"
    ].values[0]
    if status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo-64"
    ].values[0]
    != "ok"
    else row["highs-hipo-ipm"] / row["highs-hipo-64"],
    axis=1,
)

speedup_df["hipo-no2hop-speedup"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo-no2hop"
    ].values[0]
    if status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo-no2hop"
    ].values[0]
    != "ok"
    else row["highs-hipo-ipm"] / row["highs-hipo-no2hop"],
    axis=1,
)

# Rename columns for clarity
speedup_df = speedup_df.rename(
    columns={
        "highs-hipo-ipm": "ipm-time",
        "highs-hipo-64": "hipo-64-time",
        "highs-hipo-no2hop": "hipo-no2hop-time",
    }
)

# Add num-vars column by looking up in hipo_benchmarks
speedup_df["num-vars"] = speedup_df["bench-size"].apply(
    lambda bench_size: hipo_benchmarks["benchmarks"][bench_size]["Sizes"][0][
        "Num. variables"
    ]
)

# Format the dataframe for pretty printing
speedup_df = speedup_df.sort_values("num-vars")
display_df = pd.DataFrame(speedup_df["bench-size"])
display_df["num-vars"] = speedup_df["num-vars"].apply(lambda x: f"{x:,.0f}")
display_df["ipm-time"] = speedup_df["ipm-time"].apply(lambda x: naturaldelta(x))
display_df["hipo-64-time"] = speedup_df["hipo-64-time"].apply(lambda x: naturaldelta(x))
display_df["hipo-no2hop-time"] = speedup_df["hipo-no2hop-time"].apply(
    lambda x: naturaldelta(x)
)
display_df["hipo-64-speedup"] = speedup_df["hipo-64-speedup"].apply(
    lambda x: f"{x:.1f}x" if isinstance(x, (int, float)) else x
)
display_df["hipo-no2hop-speedup"] = speedup_df["hipo-no2hop-speedup"].apply(
    lambda x: f"{x:.1f}x" if isinstance(x, (int, float)) else x
)
display_df

,bench-size,num-vars,ipm-time,hipo-64-time,hipo-no2hop-time,hipo-64-speedup,hipo-no2hop-speedup
1,pypsa-eur-elec-op-7-1h,"1,646,661",3 hours,3 hours,28 minutes,1.2x,8.4x
5,pypsa-eur-elec-trex-7-1h,"1,646,674",4 hours,2 hours,29 minutes,1.9x,9.1x
2,pypsa-eur-elec-op-8-1h,"1,830,590",4 hours,2 hours,38 minutes,1.8x,7.7x
6,pypsa-eur-elec-trex-8-1h,"1,830,602",6 hours,2 hours,36 minutes,2.3x,10.3x
3,pypsa-eur-elec-op-9-1h,"2,040,798",6 hours,3 hours,33 minutes,2.2x,11.5x
7,pypsa-eur-elec-trex-9-1h,"2,040,811",7 hours,3 hours,41 minutes,2.3x,10.6x
0,pypsa-eur-elec-op-10-1h,"2,251,004",6 hours,2 hours,28 minutes,2.9x,warning
4,pypsa-eur-elec-trex-10-1h,"2,251,019",8 hours,4 hours,52 minutes,1.8x,10.2x


### Performance scaling of HiPO on a PyPSA-DE benchmark
Run ID: `20251219-pypsa-sizes`

In [64]:
results, variability = load_results(["../results/gcp-results/20251219-pypsa-sizes//"])

Found 33 records, 11 benchmark instances


In [65]:
# Create a speedup dataframe from the results dataframe
speedup_df = results.pivot_table(
    index="bench-size", columns="Solver", values="Runtime (s)", aggfunc="first"
).reset_index()

# Also pivot Status column
status_df = results.pivot_table(
    index="bench-size", columns="Solver", values="Status", aggfunc="first"
).reset_index()

# Calculate speedups relative to ipm-time, but use status if not "ok"
speedup_df["ipm-speedup"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-ipm"
    ].values[0]
    if status_df.loc[status_df["bench-size"] == row["bench-size"], "highs-ipm"].values[
        0
    ]
    != "ok"
    else row["highs"] / row["highs-ipm"],
    axis=1,
)

speedup_df["hipo-speedup"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo"
    ].values[0]
    if status_df.loc[status_df["bench-size"] == row["bench-size"], "highs-hipo"].values[
        0
    ]
    != "ok"
    else row["highs"] / row["highs-hipo"],
    axis=1,
)

# Rename columns for clarity
speedup_df = speedup_df.rename(
    columns={
        "highs": "simplex-time",
        "highs-ipm": "ipm-time",
        "highs-hipo": "hipo-time",
    }
)

# Add num-vars column by looking up in new_pypsa_benchs
speedup_df["num-vars"] = speedup_df.apply(
    lambda row: new_pypsa_benchs.loc[
        "-".join(row["bench-size"].split("-")[:-2]), "Num. variables"
    ],
    axis=1,
)

# Format the dataframe for pretty printing
speedup_df = speedup_df.sort_values("num-vars")
display_df = pd.DataFrame(speedup_df["bench-size"])
display_df["num-vars"] = speedup_df["num-vars"].apply(lambda x: f"{x:,.0f}")
display_df["simplex-time"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs"
    ].values[0]
    if status_df.loc[status_df["bench-size"] == row["bench-size"], "highs"].values[0]
    != "ok"
    else naturaldelta(row["simplex-time"]),
    axis=1,
)
display_df["ipm-time"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-ipm"
    ].values[0]
    if status_df.loc[status_df["bench-size"] == row["bench-size"], "highs-ipm"].values[
        0
    ]
    != "ok"
    else naturaldelta(row["ipm-time"]),
    axis=1,
)
display_df["hipo-time"] = speedup_df.apply(
    lambda row: status_df.loc[
        status_df["bench-size"] == row["bench-size"], "highs-hipo"
    ].values[0]
    if status_df.loc[status_df["bench-size"] == row["bench-size"], "highs-hipo"].values[
        0
    ]
    != "ok"
    else naturaldelta(row["hipo-time"]),
    axis=1,
)
display_df["ipm-speedup"] = speedup_df["ipm-speedup"].apply(
    lambda x: f"{x:.1f}x" if isinstance(x, (int, float)) else x
)
display_df["hipo-speedup"] = speedup_df["hipo-speedup"].apply(
    lambda x: f"{x:.1f}x" if isinstance(x, (int, float)) else x
)
display_df
# TODO num vars

,bench-size,num-vars,simplex-time,ipm-time,hipo-time,ipm-speedup,hipo-speedup
5,pypsa-de-elec-20-24h-20-24h,"160,813",3 minutes,2 minutes,52 seconds,1.4x,3.5x
0,pypsa-de-elec-10-12h-10-12h,"176,041",a minute,2 minutes,54 seconds,0.5x,1.7x
3,pypsa-de-elec-20-12h-20-12h,"321,413",10 minutes,9 minutes,2 minutes,1.2x,4.4x
9,pypsa-de-elec-50-24h-50-24h,"360,415",TO,11 minutes,10 minutes,5.4x,5.9x
2,pypsa-de-elec-10-3h-10-3h,"703,831",29 minutes,33 minutes,6 minutes,0.9x,4.3x
7,pypsa-de-elec-50-12h-50-12h,"720,305",TO,49 minutes,10 minutes,1.2x,5.9x
6,pypsa-de-elec-20-3h-20-3h,"1,285,013",5 hours,an hour,13 minutes,3.1x,26.3x
1,pypsa-de-elec-10-1h-10-1h,"2,111,271",8 hours,3 hours,35 minutes,2.1x,14.1x
10,pypsa-de-elec-50-3h-50-3h,"2,879,645",TO,TO,an hour,TO,6.6x
4,pypsa-de-elec-20-1h-20-1h,"3,854,613",TO,TO,2 hours,TO,11.1x


### Performance of HiPO on the v2 benchmark set
Many Run IDs..

In [66]:
results, variability = load_results(
    [
        "../results/gcp-results/20251212-run-Ls/",
        "../results/gcp-results/20251214-rerun-1/",
        "../results/gcp-results/20251215-run-S-M/",
        "../results/gcp-results/20251216-rerun-2/",
        "../results/gcp-results/20251222-leftovers/",
        "../results/gcp-results/20251227-new-pypsa/",
        # '../results/partial-results/',
    ]
)

# Remove results from failed runs:
bad = {  # TODO automate from variability analysis?
    ("benchmark-instance-l-00", "20251212-run-Ls"),
    ("benchmark-instance-l-00", "20251214-rerun-1"),
    ("benchmark-instance-l-01", "20251212-run-Ls"),
    ("benchmark-instance-l-01", "20251214-rerun-1"),
    ("benchmark-instance-s-m-01", "20251215-run-S-M"),
    ("benchmark-instance-s-m-02", "20251215-run-S-M"),
    ("benchmark-instance-s-m-06", "20251215-run-S-M"),
    ("benchmark-instance-s-m-08", "20251215-run-S-M"),
    ("benchmark-instance-s-m-12", "20251215-run-S-M"),
    ("benchmark-instance-s-m-14", "20251215-run-S-M"),
}
keys = pd.MultiIndex.from_frame(results[["Hostname", "Run ID"]])
results = results.loc[~keys.isin(bad)].copy()
print(
    f"After dropping: {len(results)} records, {len(results['bench-size'].unique())} benchmark instances"
)

Found 2577 records, 159 benchmark instances
After dropping: 2252 records, 156 benchmark instances


In [67]:
benchmarks_df = load_benchmark_metadata()

In [ ]:
# TODO
# HiPO and IPM were only run on the LPs so filter results to those
# S & M benchs were run on all years' solvers, while L was only run on latest

# Compare: HiGHS default (simplex) vs IPM vs HiPO
# Overall number of problems solved, then breakdown in each size category, model framework, etc
# SGM runtime among problems solved to get an idea of average speedup
# Largest problem solvable by each algorithm, to get an idea of how much HiPO pushes the frontier of solvable models
# Gap between HiPO and Gurobi in terms of problems solved, largest problem solvable, relative runtime on largest problems

# Are there clear recommendations in terms of when to use simplex vs IPM vs HiPO? E.g. PyPSA models, or infrastructure models?